In [21]:
from pathlib import Path
import re

# Define the Matchers directory
matchers_dir = Path("./matchers")

# Find all .py files containing "TODO"
todo_files = [f for f in matchers_dir.glob("**/*.py") if "TODO" in f.read_text()]

# Display the files found
for file in todo_files:
    print(file)

matchers\new_plugin_with_types\crates\plugins\Cargo.toml.py
matchers\new_plugin_with_types\crates\plugins\src\lib.rs.py
matchers\new_tool\Cargo.toml.py
matchers\new_tool\crates\plugins\Cargo.toml.py
matchers\new_tool\crates\plugins\src\lib.rs.py


In [28]:
def extract_commented_template(file_path):
    content = file_path.read_text()
    template_content = ""
    for line in content.splitlines():
        if line.startswith("#"):
            # template_content += line[1:] + "\n"  # Remove the "#" and add the line
            template_content += line + "\n"  # Remove the "#" and add the line
        else:
            break  # Stop reading once non-commented content is reached
    return template_content.strip()


In [29]:
def extract_variables(template_content):
    # Regular expression to find all instances of {{variable_name}}
    pattern = re.compile(r'\{\{(\w+)\}\}')
    # Find all matches and return them as a set to ensure uniqueness
    return set(pattern.findall(template_content))
def find_matching_implementation(todo_file):
    todo_template_content = extract_commented_template(todo_file)
    todo_variables = extract_variables(todo_template_content)

    for file in matchers_dir.glob("**/*.py"):
        print("Checking file:", file)
        if file not in todo_files and "TODO" not in file.read_text():
            # Extract variables from the current file being checked
            file_template_content = extract_commented_template(file)
            file_variables = extract_variables(file_template_content)

            # Compare the sets of variables
            if todo_variables == file_variables:
                return file
    return None

In [30]:
def extract_implementation_content(file_path):
    content = file_path.read_text()
    implementation_content = ""
    past_comments = False
    for line in content.splitlines():
        if past_comments or not line.startswith("#"):
            past_comments = True
            implementation_content += line + "\n"
    return implementation_content.strip()


In [31]:
todo_files.sort(key=lambda x: len(x.parts))

In [32]:
for todo_file in todo_files:
    matching_file = find_matching_implementation(todo_file)
    if matching_file:
        # Keep the template header from the todo file
        todo_template_content = extract_commented_template(todo_file)

        # Extract the working code from the match
        matching_implementation_content = extract_implementation_content(matching_file)
                
        # Combine the new template comments with the TODO file's implementation content
        final_content = todo_template_content + "\n\n" + matching_implementation_content
        
        # Write the combined content back to the TODO file
        todo_file.write_text(final_content)
        print(f"Updated: {todo_file} with template from {matching_file}")
    else:
        print(f"No matching implementation found for {todo_file}")


Checking file: matchers\new_plugin_with_types\Cargo.toml.py
Updated: matchers\new_tool\Cargo.toml.py with template from matchers\new_plugin_with_types\Cargo.toml.py
Checking file: matchers\new_plugin_with_types\Cargo.toml.py
Checking file: matchers\new_plugin_with_types\crates\plugins\Cargo.toml.py
Checking file: matchers\new_plugin_with_types\crates\plugins\src\lib.rs.py
Checking file: matchers\new_plugin_with_types\crates\{{crate_name}}\Cargo.toml.py
Checking file: matchers\new_plugin_with_types\crates\{{crate_name}}\src\lib.rs.py
Checking file: matchers\new_plugin_with_types\crates\{{crate_name}}\src\{{crate_name}}_types.rs.py
Checking file: matchers\new_plugin_with_types\crates\{{crate_name}}\src\{{crate_name}}_types_plugin.rs.py
Checking file: matchers\new_plugin_with_types\crates\{{crate_name}}_types\Cargo.toml.py
Checking file: matchers\new_plugin_with_types\crates\{{crate_name}}_types\src\lib.rs.py
Checking file: matchers\new_plugin_with_types\crates\{{crate_name}}_types\src\{{